In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
price_and_demand = pd.read_csv("price_and_demand.csv")
price_and_demand

,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,VIC1,1/11/2022 0:30,4178.18,8.94,TRADE
1,VIC1,1/11/2022 1:00,4086.02,0.14,TRADE
2,VIC1,1/11/2022 1:30,4033.37,0.02,TRADE
3,VIC1,1/11/2022 2:00,3985.64,0.00,TRADE
4,VIC1,1/11/2022 2:30,3885.64,0.02,TRADE
...,...,...,...,...,...
8347,VIC1,23/04/2023 22:00,4717.55,84.09,TRADE
8348,VIC1,23/04/2023 22:30,4520.39,81.25,TRADE
8349,VIC1,23/04/2023 23:00,4472.49,93.97,TRADE
8350,VIC1,23/04/2023 23:30,4621.36,83.96,TRADE


In [3]:
# Check datatype of the dataframe
price_and_demand.dtypes

REGION             object
SETTLEMENTDATE     object
TOTALDEMAND       float64
RRP               float64
PERIODTYPE         object
dtype: object

In [4]:
# check if there is any mising data
price_and_demand.isna().sum()

REGION            0
SETTLEMENTDATE    0
TOTALDEMAND       0
RRP               0
PERIODTYPE        0
dtype: int64

In [5]:
# Convert "SETTLEMENTDATE" column to datetime
price_and_demand["SETTLEMENTDATE"] = pd.to_datetime(price_and_demand["SETTLEMENTDATE"], format="%d/%m/%Y %H:%M")

In [6]:
# Extract date from "SETTLEMENTDATE" and create a new 'Date' column
price_and_demand['Date'] = price_and_demand['SETTLEMENTDATE'].dt.date

In [7]:
price_and_demand.sample(20)

,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE,Date
4337,VIC1,2023-01-30 09:00:00,4795.02,-0.49,TRADE,2023-01-30
2902,VIC1,2022-12-31 11:30:00,3153.92,-1.65,TRADE,2022-12-31
1346,VIC1,2022-11-29 01:30:00,3813.37,119.50,TRADE,2022-11-29
4120,VIC1,2023-01-25 20:30:00,5732.06,119.97,TRADE,2023-01-25
1122,VIC1,2022-11-24 09:30:00,4554.82,108.88,TRADE,2022-11-24
1741,VIC1,2022-12-07 07:00:00,4899.77,94.67,TRADE,2022-12-07
3149,VIC1,2023-01-05 15:00:00,3557.59,-54.20,TRADE,2023-01-05
7318,VIC1,2023-04-02 11:30:00,2725.68,-1.90,TRADE,2023-04-02
1936,VIC1,2022-12-11 08:30:00,3192.44,-38.00,TRADE,2022-12-11
1746,VIC1,2022-12-07 09:30:00,3929.65,-19.90,TRADE,2022-12-07


In [8]:
# Calculate maximum daily energy demand and average rrp
max_edemand = price_and_demand.groupby("Date")['TOTALDEMAND'].max()
mean_rrp = price_and_demand.groupby("Date")['RRP'].mean()

In [9]:
weather = pd.read_csv("weather.csv", encoding='latin-1')
weather

,Location,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,"Melbourne (Olympic Park), Victoria",1/11/2022,8.5,13.3,3.0,NaN,NaN,SW,44.0,11:36,...,NaN,NW,13,991.3,13.2,59.0,NaN,WNW,11.0,991.5
1,"Melbourne (Olympic Park), Victoria",2/11/2022,6.9,15.7,2.8,NaN,NaN,SSW,43.0,11:33,...,NaN,W,9,1006.7,15.1,54.0,NaN,SSW,13.0,1008.4
2,"Melbourne (Olympic Park), Victoria",3/11/2022,9.1,15.4,0.4,NaN,NaN,SSW,31.0,8:07,...,NaN,SSW,9,1019.2,13.8,67.0,NaN,SSW,9.0,1021.3
3,"Melbourne (Olympic Park), Victoria",4/11/2022,10.2,17.8,0.2,NaN,NaN,S,24.0,14:50,...,NaN,WSW,6,1028.1,16.9,56.0,NaN,SW,9.0,1026.6
4,"Melbourne (Olympic Park), Victoria",5/11/2022,11.8,22.7,0.0,NaN,NaN,N,31.0,11:58,...,NaN,N,9,1026.3,18.8,65.0,NaN,SW,11.0,1023.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,"Melbourne (Olympic Park), Victoria",20/04/2023,7.8,18.5,0.0,NaN,NaN,SSW,17.0,12:47,...,NaN,NNE,7,1026.8,17.5,58.0,NaN,SW,4.0,1025.5
171,"Melbourne (Olympic Park), Victoria",21/04/2023,11.3,19.0,0.0,NaN,NaN,SSW,13.0,10:48,...,NaN,NE,6,1030.5,18.6,64.0,NaN,SSE,2.0,1029.2
172,"Melbourne (Olympic Park), Victoria",22/04/2023,14.6,19.2,0.0,NaN,NaN,SSW,22.0,16:09,...,NaN,SW,6,1031.9,18.8,64.0,NaN,SSW,11.0,1029.7
173,"Melbourne (Olympic Park), Victoria",23/04/2023,14.8,19.0,0.0,NaN,NaN,SSW,26.0,15:45,...,NaN,SSE,7,1034.4,17.8,71.0,NaN,SSW,13.0,1032.4


In [10]:
weather.dtypes

Location                              object
Date                                  object
Minimum temperature (°C)             float64
Maximum temperature (°C)             float64
Rainfall (mm)                        float64
Evaporation (mm)                     float64
Sunshine (hours)                     float64
Direction of maximum wind gust        object
Speed of maximum wind gust (km/h)    float64
Time of maximum wind gust             object
9am Temperature (°C)                 float64
9am relative humidity (%)              int64
9am cloud amount (oktas)             float64
9am wind direction                    object
9am wind speed (km/h)                 object
9am MSL pressure (hPa)               float64
3pm Temperature (°C)                 float64
3pm relative humidity (%)            float64
3pm cloud amount (oktas)             float64
3pm wind direction                    object
3pm wind speed (km/h)                float64
3pm MSL pressure (hPa)               float64
dtype: obj

In [11]:
weather.isna().sum()

Location                               0
Date                                   0
Minimum temperature (°C)               0
Maximum temperature (°C)               1
Rainfall (mm)                          0
Evaporation (mm)                     175
Sunshine (hours)                     175
Direction of maximum wind gust         1
Speed of maximum wind gust (km/h)      1
Time of maximum wind gust              1
9am Temperature (°C)                   0
9am relative humidity (%)              0
9am cloud amount (oktas)             175
9am wind direction                     0
9am wind speed (km/h)                  0
9am MSL pressure (hPa)                 0
3pm Temperature (°C)                   1
3pm relative humidity (%)              1
3pm cloud amount (oktas)             175
3pm wind direction                     1
3pm wind speed (km/h)                  1
3pm MSL pressure (hPa)                 1
dtype: int64

In [12]:
missing_value = weather[["Minimum temperature (°C)", "Maximum temperature (°C)", "Rainfall (mm)", "Speed of maximum wind gust (km/h)"]] .isnull()
missing_value

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Speed of maximum wind gust (km/h)
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
170,False,False,False,False
171,False,False,False,False
172,False,False,False,False
173,False,False,False,False


In [13]:
weather["Date"] = pd.to_datetime(weather['Date'], format="%d/%m/%Y")
weather.head()

,Location,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,"Melbourne (Olympic Park), Victoria",2022-11-01,8.5,13.3,3.0,NaN,NaN,SW,44.0,11:36,...,NaN,NW,13,991.3,13.2,59.0,NaN,WNW,11.0,991.5
1,"Melbourne (Olympic Park), Victoria",2022-11-02,6.9,15.7,2.8,NaN,NaN,SSW,43.0,11:33,...,NaN,W,9,1006.7,15.1,54.0,NaN,SSW,13.0,1008.4
2,"Melbourne (Olympic Park), Victoria",2022-11-03,9.1,15.4,0.4,NaN,NaN,SSW,31.0,8:07,...,NaN,SSW,9,1019.2,13.8,67.0,NaN,SSW,9.0,1021.3
3,"Melbourne (Olympic Park), Victoria",2022-11-04,10.2,17.8,0.2,NaN,NaN,S,24.0,14:50,...,NaN,WSW,6,1028.1,16.9,56.0,NaN,SW,9.0,1026.6
4,"Melbourne (Olympic Park), Victoria",2022-11-05,11.8,22.7,0.0,NaN,NaN,N,31.0,11:58,...,NaN,N,9,1026.3,18.8,65.0,NaN,SW,11.0,1023.2


In [14]:
# Extract selected variable columns to form new dataframe
selected_variables = ["Date", "Minimum temperature (°C)", "Maximum temperature (°C)", "Rainfall (mm)", "Speed of maximum wind gust (km/h)"]
processed_data = weather[selected_variables].set_index('Date')

In [15]:
processed_data.head()

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Speed of maximum wind gust (km/h)
Date,,,,
2022-11-01,8.5,13.3,3.0,44.0
2022-11-02,6.9,15.7,2.8,43.0
2022-11-03,9.1,15.4,0.4,31.0
2022-11-04,10.2,17.8,0.2,24.0
2022-11-05,11.8,22.7,0.0,31.0


In [16]:
#substitude missing data with the average value of the respective column
processed_data[["Maximum temperature (°C)", "Speed of maximum wind gust (km/h)"]] = processed_data[["Maximum temperature (°C)", "Speed of maximum wind gust (km/h)"]].fillna(processed_data[["Maximum temperature (°C)", "Speed of maximum wind gust (km/h)"]].mean())
processed_data.tail()

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Speed of maximum wind gust (km/h)
Date,,,,
2023-04-20,7.8,18.500000,0.0,17.000000
2023-04-21,11.3,19.000000,0.0,13.000000
2023-04-22,14.6,19.200000,0.0,22.000000
2023-04-23,14.8,19.000000,0.0,26.000000
2023-04-24,11.0,23.121839,0.0,34.563218


In [17]:
processed_data.insert(loc = 2, column = "Average Temerature (°C)", value = (processed_data["Minimum temperature (°C)"] + processed_data["Maximum temperature (°C)"]) / 2)
processed_data.head()

,Minimum temperature (°C),Maximum temperature (°C),Average Temerature (°C),Rainfall (mm),Speed of maximum wind gust (km/h)
Date,,,,,
2022-11-01,8.5,13.3,10.90,3.0,44.0
2022-11-02,6.9,15.7,11.30,2.8,43.0
2022-11-03,9.1,15.4,12.25,0.4,31.0
2022-11-04,10.2,17.8,14.00,0.2,24.0
2022-11-05,11.8,22.7,17.25,0.0,31.0


In [18]:
processed_data["Average RRP"] = mean_rrp
processed_data["Maximum Energry Demand"] = max_edemand
processed_data.tail()

,Minimum temperature (°C),Maximum temperature (°C),Average Temerature (°C),Rainfall (mm),Speed of maximum wind gust (km/h),Average RRP,Maximum Energry Demand
Date,,,,,,,
2023-04-20,7.8,18.500000,13.15000,0.0,17.000000,130.707500,6084.21
2023-04-21,11.3,19.000000,15.15000,0.0,13.000000,131.461042,5903.60
2023-04-22,14.6,19.200000,16.90000,0.0,22.000000,80.391458,5313.61
2023-04-23,14.8,19.000000,16.90000,0.0,26.000000,63.213750,5361.47
2023-04-24,11.0,23.121839,17.06092,0.0,34.563218,63.770000,4395.96
